In this implementation, I used a bitmask representation of the puzzle to enable faster operations on numbers instead of matrices.

**Q1.**

States: Represented as a numeric value corresponding to each puzzle configuration.
Actions: Involve flipping the appropriate bits in the state..
Initial state: The bitmask representation of the input puzzle.
Goal state: The number 0, representing all lights off.

**Q2.**

*BFS:* This algorithm explores the search space level by level, making it both *optimal* and *complete*.

*IDS:* This algorithm combines the level-by-level search of BFS with the space efficiency of DFS. By avoiding loops during the depth search, the algorithm is *complete*, and its level-by-level approach ensures it is *optimal* as well.

*A Star:* This algorithm is a combination of UCS and Greedy Best First Seach. It uses fringe (which in this case, is a priority queue) to extract the node which has the minimum *f* score (sum of heuristic of the node and the actual cost to that node so far). As it is explained in the class, this algorithm would be *optimal* in a tree search if the given heuristic is admissible, and *optimal* in a graph seatch, if the given heuristic is consistent.

**Q3.**

**Heuristics.**
One simple heuristic is the number of lights that are on. However, it is neither admissible nor consistent. For example, consider a puzzle position where one light and its neighbors are on. This puzzle can be solved in a single move by turning off the center light, but the heuristic value would be $4$, making it inadmissible.

Another heuristic is just $\frac{1}{5}$ of the previous one, and this version is consistent. For example, consider a puzzle A and its successor $B$ after a move $m$. In the worst case, $m$ turns on $5$ lights, so the difference in heuristic values between the two nodes would be $\frac{5}{5} = 1$, which matches the cost of any move $m$. In all other cases, the change in heuristic values, $\Delta$, would be less than $1$, making this heuristic consistent.

A nonsensical heuristic is the following function:
$h(pzl)=Σx,y​(pzl[x][y] \;\; =? \;\; 1)$.

There's no logical reasoning behind this heuristic, but strangely enough, it works the best!


The other file contains the same problem and algorithms as the solution, but with one key difference: the puzzle is implemented using a NumPy array (matrix). This approach allowed me to account for certain symmetric cases in the puzzle, like rotations and diagonal transpositions. While it reduces some redundancy during the solving process, it significantly slows down the algorithm due to the high overhead involved in checking for these repetitions.

In [1]:
import random
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Callable, Any, Set
import numpy as np
from time import time
from tabulate import tabulate
import threading
import collections
from math import ceil
from enum import Enum
from functools import partial
from copy import deepcopy
import sys

In [2]:
class LightsOutPuzzle_BITMASK:
    def __init__(self, board: List[List[int]]):
        self.size = len(board)
        self.mask_size = self.size * self.size
        self.board = np.array(board, dtype=int)
        self.bitmask = self.board_to_bitmask()

    def board_to_bitmask(self) -> int:
        """Convert the board to a bitmask."""
        bitmask = 0
        for row in self.board:
            for cell in row:
                bitmask = (bitmask << 1) | cell
        return bitmask

    def toggle(self, x, y):
        """Toggle the cell at (x, y) and its neighbors."""
        def pos_to_mask(x, y):
            """Get the bitmask for a specific cell."""
            return 1 << (self.mask_size - 1 - (x * self.size + y))

        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.bitmask ^= pos_to_mask(nx, ny)

    def is_solved(self):
        """Check if the puzzle is solved (all bits are 0)."""
        return self.bitmask == 0

    def get_moves(self):
        """Return all possible (x, y) moves."""
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __hash__(self):
        return self.bitmask

    def __eq__(self, other):
        return self.bitmask == other.bitmask

    def __lt__(self, other):
        return bin(self.bitmask).count('1') < bin(other.bitmask).count('1')

    def __str__(self):
        """Convert the bitmask back to the board format for display."""
        return '\n'.join(' '.join(str((self.bitmask >> (self.mask_size - 1 - (x * self.size + y))) & 1)
                                  for y in range(self.size)) for x in range(self.size))

class LightsOutPuzzle_TEST:
    def __init__(self, board: List[List[int]]):
        self.board = np.array(board, dtype=np.uint8)
        self.size = len(board)

    def toggle(self, x, y):
        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.board[nx, ny] ^= 1

    def is_solved(self):
        return np.all(self.board == 0)

    def get_moves(self):
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)

LightsOutPuzzle = LightsOutPuzzle_BITMASK

In [3]:
def show_solution(
    puzzle: LightsOutPuzzle,
    solution: List[Tuple[int, int]],
    algorithm_name: str,
    nodes_visited: int,
    show_steps: bool = False,
):
    print(f"\nSolving with {algorithm_name}:")
    if solution is None:
        print("No solution found.")
        return
    print(f"Solution: {solution}")
    print(f"Nodes visited: {nodes_visited}")
    if show_steps:
        print("\nSolution steps:")
        for i, move in enumerate(solution):
            print(f"\nStep {i+1}: Toggle position {move}")
            puzzle.toggle(*move)
            print(puzzle)
        print(
            "\nFinal state (solved):"
            if puzzle.is_solved()
            else "\nFinal state (not solved):"
        )
        print(puzzle)

In [4]:
def create_random_board(size: int,  seed: int, num_toggles: int = None):
    random.seed(time() if seed is None else seed)

    board = [[0 for _ in range(size)] for _ in range(size)]
    puzzle = LightsOutPuzzle_TEST(board)

    if num_toggles is None:
        num_toggles = random.randint(1, size * size)

    for _ in range(num_toggles):
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        puzzle.toggle(x, y)

    return puzzle.board

tests = [
    create_random_board(3, 42),
    create_random_board(3, 41),
    create_random_board(3, 42, 5),
    create_random_board(4, 42),
    create_random_board(4, 41),
    create_random_board(4, 42, 5),
    create_random_board(5, 42),
    create_random_board(5, 41),
    create_random_board(5, 42, 5),
]

In [5]:
def bfs_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    queue = collections.deque()
    queue.append([puzzle, []])

    visited = set()
    visited.add(puzzle.bitmask)
    cnt = 0
    while queue:
        curr, mvs = queue.popleft()
        if curr.is_solved():
            return (mvs, cnt)

        for mv in curr.get_moves():
            pzl = LightsOutPuzzle([[0]*puzzle.size for _ in range(puzzle.size)])  
            pzl.bitmask = curr.bitmask  
            pzl.toggle(*mv)

            if pzl.bitmask not in visited:
                visited.add(pzl.bitmask)
                cnt += 1
                queue.append((pzl, mvs + [mv]))

    return (None, cnt)

pzl = LightsOutPuzzle([
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 0, 0, 0]
])
pzl = LightsOutPuzzle(deepcopy(tests[-1]))
start_time = time()
moves, nodes_visited = bfs_solve(pzl)
print(f"Moves: {moves}")
print(f"Nodes Visited: {nodes_visited}")
print(f"Time taken: {time() - start_time:.4f} seconds")

Moves: [(0, 0), (1, 1), (2, 1)]
Nodes Visited: 3742
Time taken: 0.1278 seconds


In [6]:
def ids_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    cnt = 0
    def depth_search(puzzle: LightsOutPuzzle, dpth: int, path: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
        nonlocal cnt
        if puzzle.is_solved():
            return path

        if dpth == 0:
            return None

        for mv in puzzle.get_moves():
            pzl = LightsOutPuzzle([[0]*puzzle.size for _ in range(puzzle.size)]) 
            pzl.bitmask = puzzle.bitmask
            pzl.toggle(*mv)

            cnt += 1
            pth = depth_search(pzl, dpth - 1, path + [mv])
            if pth is not None:
                return pth
        return None

    mx_dpth = 0
    while mx_dpth < puzzle.size * puzzle.size:
        path = depth_search(puzzle, mx_dpth, [])

        if path is not None:
            return (path, cnt)

        mx_dpth += 1
    return (None, cnt)

pzl = LightsOutPuzzle([
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 0, 0, 0]
])
pzl = LightsOutPuzzle(deepcopy(tests[-1]))

start_time = time()
path, nodes_visited = ids_solve(pzl)
print(f"Solution Path: {path}")
print(f"Nodes Visited: {nodes_visited}")
print(f"Time taken: {time() - start_time:.4f} seconds")

Solution Path: [(0, 0), (1, 1), (2, 1)]
Nodes Visited: 845
Time taken: 0.0156 seconds


In [7]:
def heuristic1(puzzle : LightsOutPuzzle) -> int:
    return bin(puzzle.bitmask).count('1')

def heuristic2(puzzle : LightsOutPuzzle) -> int:
    return bin(puzzle.bitmask).count('1') / 5

def manhattan_distance_heuristic(puzzle: LightsOutPuzzle) -> int:
    size = puzzle.size
    total_distance = 0
    for x in range(size):
        for y in range(size):
            if (puzzle.bitmask >> (puzzle.mask_size - 1 - (x * size + y)) & 1) == 1:  
                total_distance += (x+y)
    return total_distance

heuristics = [heuristic1, heuristic2, manhattan_distance_heuristic]

In [8]:
def weighted_heuristic1(puzzle : LightsOutPuzzle, alpha = 5):
  return heuristic1(puzzle) * alpha

def weighted_heuristic2(puzzle : LightsOutPuzzle, alpha = 5):
  return heuristic2(puzzle) * alpha

def weighted_heuristic3(puzzle : LightsOutPuzzle, alpha = 5):
  return manhattan_distance_heuristic(puzzle) * alpha

weighted_heuristics = [weighted_heuristic1, weighted_heuristic2, weighted_heuristic3]
weights = [2, 5, 10]

In [9]:
def astar_solve(puzzle: LightsOutPuzzle, heuristic: Callable[[LightsOutPuzzle], int], cnt_flag = False) -> Tuple[List[Tuple[int, int]], int]:
    open_set = []
    heapq.heappush(open_set, (0, 0, puzzle, []))
    g_score = {puzzle.bitmask: 0}
    f_score = {puzzle.bitmask: heuristic(puzzle)}
    cnt = 0

    while open_set:
        if cnt_flag and cnt % 5000 == 0:
            sys.stdout.write(f"\rCurrent cnt: {cnt}")
            sys.stdout.flush()
        _, _, current, path = heapq.heappop(open_set)

        if current.is_solved():
            return (path, cnt)

        for move in current.get_moves():
            neighbor = LightsOutPuzzle([[0]*current.size for _ in range(current.size)]) 
            neighbor.bitmask = current.bitmask 
            neighbor.toggle(*move)

            tentative_g_score = g_score[current.bitmask] + 1
            if neighbor.bitmask not in g_score or tentative_g_score < g_score[neighbor.bitmask]:
                g_score[neighbor.bitmask] = tentative_g_score
                f_score[neighbor.bitmask] = tentative_g_score + heuristic(neighbor)
                heapq.heappush(open_set, (f_score[neighbor.bitmask], tentative_g_score, neighbor, path + [move]))
                cnt += 1

    return (None, cnt)

pzl = LightsOutPuzzle([
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 0, 0, 0]
])
pzl = LightsOutPuzzle(deepcopy(tests[-3]))


print(pzl, end="\n\n")
start_time = time()
p, c = astar_solve(pzl, weighted_heuristic1, cnt_flag = True)
print("\nPath:", p)
print("Nodes Visited:", c)
print(f"Time taken: {time() - start_time:.4f} seconds")

pzl = deepcopy(pzl)
for mv in p:
    pzl.toggle(*mv)
print(f"Is solved: {pzl.is_solved()}", end='\n\n')
print(pzl, end='\n\n')

0 0 0 0 1
1 0 0 1 0
0 0 0 1 1
0 1 0 1 1
1 1 0 1 0

Current cnt: 0


Path: [(2, 3), (4, 1), (4, 3), (3, 4), (1, 4), (1, 3), (2, 2), (3, 1), (4, 0), (3, 0), (2, 0), (0, 2), (1, 1), (0, 1), (0, 0)]
Nodes Visited: 5731
Time taken: 0.1031 seconds
Is solved: True

0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0



In [10]:
class SearchStatus(Enum):
    SOLVED = "Solved"
    TIMEOUT = "Timeout"
    NO_SOLUTION = "No Solution"

@dataclass
class SearchResult:
    result: SearchStatus
    solution: Any = None
    nodes_visited: int = None
    time: float = None

    def __str__(self):
        output = f"Result: {self.result.value}\n"
        if self.result == SearchStatus.SOLVED:
            output += f"Solution: {self.solution}\n"
        if self.result != SearchStatus.TIMEOUT:
            output += f"Nodes Visited: {self.nodes_visited}\n"
            output += f"Time Taken: {self.time:.3f} seconds\n"

        return output


DEFAULT_SEARCH_RESULT = SearchResult(SearchStatus.TIMEOUT, None, None, float("inf"))

In [11]:
def run_searches(func: Callable, args: Any, time_limit: float = 60.0, name: str = "") -> SearchResult:
    result = []

    def target():
        try:
            solution, nodes_visited = func(*args)
            result.append((solution, nodes_visited))
        except Exception as e:
            result.append(e)

    thread = threading.Thread(target=target)
    thread.start()

    start_time = time()
    thread.join(timeout=time_limit)

    if thread.is_alive() or not result:
        print(f"\nTime limit of {time_limit} seconds exceeded for {name}")
        return SearchResult(SearchStatus.TIMEOUT)

    if isinstance(result[0], Exception):
        raise result[0]

    solution, nodes_visited = result[0]
    show_solution(args[0], solution, name, nodes_visited)
    time_taken = time() - start_time
    return SearchResult(SearchStatus.SOLVED if solution else SearchStatus.NO_SOLUTION, solution, nodes_visited, time_taken)

In [12]:
def run_test(test: List, heuristics: List[Callable[[Any], int]], time_limit: float = 60.0):

    print(f"Running test:\n {test}")

    puzzle = LightsOutPuzzle(test)

    bfs_result = run_searches(bfs_solve, (deepcopy(puzzle),), name="BFS", time_limit=time_limit)
    ids_result = run_searches(ids_solve, (deepcopy(puzzle),), name="IDS", time_limit=time_limit)

    astar_results = []
    for heuristic in heuristics:
        astar_results.append(
            run_searches(
                astar_solve,
                (deepcopy(puzzle), heuristic),
                name=f"A*({heuristic.__name__})",
                time_limit=time_limit,
            )
        )

    weighted_astar_results = []
    for heuristic in weighted_heuristics:
        for weight in weights:
          weighted_astar_results.append(
              run_searches(
                  astar_solve,
                  (deepcopy(puzzle), partial(heuristic, alpha = weight)),
                  name=f"weighted A*({heuristic.__name__}, alpha = {weight})",
                  time_limit=time_limit,
              )
          )

    print("\n-------------------------------\n")

    return (
        test,
        bfs_result,
        ids_result,
        *astar_results,
        *weighted_astar_results
    )

In [13]:
results: List[Tuple[str, SearchResult, SearchResult]] = []

for test in tests:
    results.append(run_test(test, heuristics, 120))

Running test:
 [[1 1 1]
 [1 1 1]
 [1 0 0]]

Solving with BFS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 94

Solving with IDS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 34

Solving with A*(heuristic1):
Solution: [(1, 0), (0, 2)]
Nodes visited: 24

Solving with A*(heuristic2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 42

Solving with A*(manhattan_distance_heuristic):
Solution: [(0, 2), (1, 0)]
Nodes visited: 17

Solving with weighted A*(weighted_heuristic1, alpha = 2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 38

Solving with weighted A*(weighted_heuristic1, alpha = 5):
Solution: [(1, 0), (0, 2)]
Nodes visited: 38

Solving with weighted A*(weighted_heuristic1, alpha = 10):
Solution: [(1, 0), (0, 2)]
Nodes visited: 38

Solving with weighted A*(weighted_heuristic2, alpha = 2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 24

Solving with weighted A*(weighted_heuristic2, alpha = 5):
Solution: [(1, 0), (0, 2)]
Nodes visited: 24

Solving with weighted A*(weighted_heuristic2, alpha = 10):
Solut

In [14]:
table_data: List[List[str]] = []

for result in results:
    table_data.append(
        [
            "\n".join(" ".join(str(cell) for cell in row) for row in result[0]),
            str(result[1]),
            str(result[2]),
        ]
    )

    for i in range(len(heuristics)):
        table_data[-1].append(str(result[2 + i]))

    for i in range(len(weighted_heuristics)):
        for j in range(len(weights)):
          table_data[-1].append(str(result[2 + len(heuristics) + i * 2 + j]))

    table_data.append("\n\n\n")

if table_data[-1] == "\n\n\n":
    table_data.pop()

print(len(table_data))
print("Results:")
print(
    tabulate(
        table_data,
        headers=[
            "Test",
            "BFS",
            "IDS",
            *[f"A* ({heuristic.__name__})" for heuristic in heuristics],
            *[f"Weighted A* ({heuristic.__name__}, alpha = {weight})" for heuristic in weighted_heuristics for weight in weights]
        ],
        floatfmt=".3f",
        numalign="center",
        stralign="center",
        tablefmt="grid",
    )
)

17
Results:
+-----------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------